In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    trained_mdeol_path: Path
    training_data: Path
    params_batch_size: int
    params_image_size: list

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

In [4]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, 'PetImages')
        evaluation_config = EvaluationConfig(
            trained_mdeol_path=self.config.training.trained_model_path,
            training_data=training_data,
            params_batch_size=self.params.BATCH_SIZE,
            params_image_size=self.params.IMAGE_SIZE
        )

        return evaluation_config

In [12]:
import tensorflow as tf

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def load_model(self):
        self.model = tf.keras.models.load_model(
            self.config.trained_mdeol_path
        )

    def _evaluation_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )

        evaluation_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.evaluation_generator = evaluation_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    def evaluate(self):
        self.load_model()
        self._evaluation_generator()
        self.score = self.model.evaluate(self.evaluation_generator)
        save_json(
            path=Path('score.json'),
            data={
                'loss': self.score[0],
                'accuracy': self.score[1]
            }
        )

In [13]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(evaluation_config)
    evaluation.load_model()
    evaluation.evaluate()
except Exception as e:
    raise e

[2022-09-25 15:29:20,084: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-25 15:29:20,087: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-25 15:29:20,088: INFO: common]: created directory at: artifacts
Found 4998 images belonging to 2 classes.
313/313 [==============================] - 25s 64ms/step - loss: 1.3851 - accuracy: 0.8737
[2022-09-25 15:29:46,264: INFO: common]: json file saved at: score.json
